<a href="https://colab.research.google.com/github/DataMinati/Super-Notebooks-1K/blob/main/Q%26A_Chatbot_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

!pip install transformers

     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 52.0 MB/s 
     |████████████████████████████████| 67 kB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 25.5 MB/s 
     |████████████████████████████████| 596 kB 52.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
coqa = pd.read_json('http://downloads.cs.stanford.edu/nlp/data/coqa/coqa-train-v1.0.json')
coqa.head()

,version,data
0,1,"{'source': 'wikipedia', 'id': '3zotghdk5ibi9ce..."
1,1,"{'source': 'cnn', 'id': '3wj1oxy92agboo5nlq4r7..."
2,1,"{'source': 'gutenberg', 'id': '3bdcf01ogxu7zdn..."
3,1,"{'source': 'cnn', 'id': '3ewijtffvo7wwchw6rtya..."
4,1,"{'source': 'gutenberg', 'id': '3urfvvm165iantk..."


In [3]:
del coqa["version"]

In [4]:
coqa['data'][0]

{'answers': [{'input_text': 'It was formally established in 1475',
   'span_end': 179,
   'span_start': 151,
   'span_text': 'Formally established in 1475',
   'turn_id': 1},
  {'input_text': 'research',
   'span_end': 494,
   'span_start': 454,
   'span_text': 'he Vatican Library is a research library',
   'turn_id': 2},
  {'input_text': 'history, and law',
   'span_end': 511,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law',
   'turn_id': 3},
  {'input_text': 'philosophy, science and theology',
   'span_end': 545,
   'span_start': 457,
   'span_text': 'Vatican Library is a research library for history, law, philosophy, science and theology',
   'turn_id': 4},
  {'input_text': 'a  project',
   'span_end': 879,
   'span_start': 769,
   'span_text': 'March 2014, the Vatican Library began an initial four-year project of digitising its collection of manuscripts',
   'turn_id': 5},
  {'input_text': 'into periods',
   'span_end': 1127,
   'span_

In [5]:
coqa['data'][0].keys()

dict_keys(['source', 'id', 'filename', 'story', 'questions', 'answers', 'name'])

In [6]:
cols = ["text","question","answer"]

In [7]:
cols = ["text","question","answer"]

comp_list = []
for index, row in coqa.iterrows():
    for i in range(len(row["data"]["questions"])):
        temp_list = []
        temp_list.append(row["data"]["story"])
        temp_list.append(row["data"]["questions"][i]["input_text"])
        temp_list.append(row["data"]["answers"][i]["input_text"])
        comp_list.append(temp_list)

new_df = pd.DataFrame(comp_list, columns=cols) 

In [8]:
new_df.to_csv("CoQA_data.csv", index=False)
data = pd.read_csv("CoQA_data.csv")
data.head()

,text,question,answer
0,"The Vatican Apostolic Library (), more commonl...",When was the Vat formally opened?,It was formally established in 1475
1,"The Vatican Apostolic Library (), more commonl...",what is the library for?,research
2,"The Vatican Apostolic Library (), more commonl...",for what subjects?,"history, and law"
3,"The Vatican Apostolic Library (), more commonl...",and?,"philosophy, science and theology"
4,"The Vatican Apostolic Library (), more commonl...",what was started in 2014?,a project


In [9]:
print("Number of question and answers: ", len(data))

Number of question and answers:  108647


In [10]:
from transformers import TFBertForQuestionAnswering
from transformers import BertTokenizer

In [11]:
model = TFBertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [12]:
random_num = np.random.randint(0,len(data))

question = data["question"][random_num]
text = data["text"][random_num]
random_num

87424

In [13]:
print('Context: \n')
print(text)
print('\nQuestion: \n')
print(question)

Context: 

Every month, Julia and her cousins would go to visit their grandparents. They would be excited because their grandfather would give them a few coins. Then they would buy toys and sweets. The grandparents commented that, behaving like this, the children would never learn to manage their money. So they planned a special test, in which the children would have to show, over the course of a year, just what they could manage to get with those few coins. 

Some thought that they would save their money, but Rubin and Nico continued spending it all on sweets. So they give up saving. 

Monty decided to manage his money by exchanging it: buying and selling things, or bettering it with others. Soon he surprised the whole family. He had accumulated lots of money. However, Monty was not very careful, and he got involved in more and more risky deals. A few months later he became penniless after a losing bet. 

Alex, had a will of iron. He saved the money and at the end of the year he had c

In [14]:
input_ids = tokenizer(question, text, return_tensors="tf")
input_ids.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
input_ids.input_ids[0]

<tf.Tensor: shape=(430,), dtype=int32, numpy=
array([  101,  2106,  2014, 13055, 14300,  1029,   102,  2296,  3204,
        1010,  6423,  1998,  2014, 12334,  2052,  2175,  2000,  3942,
        2037, 14472,  1012,  2027,  2052,  2022,  7568,  2138,  2037,
        5615,  2052,  2507,  2068,  1037,  2261,  7824,  1012,  2059,
        2027,  2052,  4965, 10899,  1998, 26844,  1012,  1996, 14472,
        7034,  2008,  1010,  2022,  3270,  6455,  2066,  2023,  1010,
        1996,  2336,  2052,  2196,  4553,  2000,  6133,  2037,  2769,
        1012,  2061,  2027,  3740,  1037,  2569,  3231,  1010,  1999,
        2029,  1996,  2336,  2052,  2031,  2000,  2265,  1010,  2058,
        1996,  2607,  1997,  1037,  2095,  1010,  2074,  2054,  2027,
        2071,  6133,  2000,  2131,  2007,  2216,  2261,  7824,  1012,
        2070,  2245,  2008,  2027,  2052,  3828,  2037,  2769,  1010,
        2021, 20524,  1998, 19332,  2506,  5938,  2009,  2035,  2006,
       26844,  1012,  2061,  2027,  2507,  2

In [16]:
print("The input has a total of {} tokens.".format(len(input_ids.input_ids[0])))

The input has a total of 430 tokens.


In [17]:
tokens = tokenizer.convert_ids_to_tokens(input_ids.input_ids[0])

for token, id in zip(tokens, input_ids.input_ids[0]):
    print('{:8}{:8,}'.format(token,id))

[CLS]        101
did        2,106
her        2,014
grandma   13,055
approve   14,300
?          1,029
[SEP]        102
every      2,296
month      3,204
,          1,010
julia      6,423
and        1,998
her        2,014
cousins   12,334
would      2,052
go         2,175
to         2,000
visit      3,942
their      2,037
grandparents  14,472
.          1,012
they       2,027
would      2,052
be         2,022
excited    7,568
because    2,138
their      2,037
grandfather   5,615
would      2,052
give       2,507
them       2,068
a          1,037
few        2,261
coins      7,824
.          1,012
then       2,059
they       2,027
would      2,052
buy        4,965
toys      10,899
and        1,998
sweets    26,844
.          1,012
the        1,996
grandparents  14,472
commented   7,034
that       2,008
,          1,010
be         2,022
##ha       3,270
##ving     6,455
like       2,066
this       2,023
,          1,010
the        1,996
children   2,336
would      2,052
never      2,196
le

In [18]:
input_ids.token_type_ids[0]    #Where 0's correpond to question tokens and 1's to context tokens.

<tf.Tensor: shape=(430,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
     

In [19]:
output = model(input_ids)

In [20]:
print(output.start_logits)
print('\n')
print(output.end_logits)

tf.Tensor(
[[ 0.20979425 -5.270901   -3.7882028  -5.5252857  -6.457049   -9.324766
   0.20978348 -3.8094811  -5.5436263  -8.255427   -2.441989   -7.824565
  -4.6334405  -5.8675356  -7.5329595  -6.7145953  -7.531072   -5.2597795
  -3.9781578  -2.7730105  -6.2683353  -2.6673949  -5.9336467  -5.950813
  -3.792148   -6.262453   -2.8177452  -4.2182364  -7.0107923  -5.931168
  -7.5410085  -7.346431   -7.5701447  -5.7338815  -8.377481   -6.347635
  -6.231136   -7.7628756  -6.198014   -6.384913   -8.615542   -5.8144455
   0.20980448 -0.10747269 -0.6257924  -2.7221313  -3.2179985  -5.796309
  -3.8736043  -6.5250254  -7.8211856  -6.8452315  -6.5080905  -5.60304
  -1.3577293  -1.6523794  -3.824323   -3.1212983  -5.356862   -7.032823
  -5.985474   -7.030347   -4.5993886  -6.241033   -4.0081453  -2.4116447
  -4.010243   -5.2370296  -3.8234365  -4.6462393  -7.9129663  -6.8317037
  -7.318166   -5.1707225  -5.217364   -7.18981    -6.9503465  -6.974129
  -5.658301   -8.18093    -6.5787444  -7.55889    

In [21]:
#tokens with highest start and end scores
answer_start = tf.argmax(tf.cast(output.start_logits, tf.int32), axis=1)
answer_end = tf.where(tf.equal(output.end_logits, float(tf.reduce_max(output.end_logits[0]))))[:,-1]

In [22]:
print(answer_start, answer_end)

tf.Tensor([0], shape=(1,), dtype=int64) tf.Tensor([42], shape=(1,), dtype=int64)


In [23]:
if answer_end >= answer_start:
    answer = " ".join(tokens[int(answer_start):int(answer_end)+1])
else:
    print("I am unable to find the answer to this question. Can you please ask another question?")
    
print("Text:\n{}".format(text.capitalize()))
print("\nQuestion:\n{}".format(question.capitalize()))
print("\nAnswer:\n{}.".format(answer.capitalize()))

Text:
Every month, julia and her cousins would go to visit their grandparents. they would be excited because their grandfather would give them a few coins. then they would buy toys and sweets. the grandparents commented that, behaving like this, the children would never learn to manage their money. so they planned a special test, in which the children would have to show, over the course of a year, just what they could manage to get with those few coins. 

some thought that they would save their money, but rubin and nico continued spending it all on sweets. so they give up saving. 

monty decided to manage his money by exchanging it: buying and selling things, or bettering it with others. soon he surprised the whole family. he had accumulated lots of money. however, monty was not very careful, and he got involved in more and more risky deals. a few months later he became penniless after a losing bet. 

alex, had a will of iron. he saved the money and at the end of the year he had collec

In [24]:
data.loc[random_num]

text        Every month, Julia and her cousins would go to...
question                             Did her grandma approve?
answer                                                     no
Name: 87424, dtype: object

In [25]:
answer = tokens[int(answer_start)]

for i in range(int(answer_start)+1, int(answer_end)+1):
    if tokens[i][0:2] == "##":
        answer += tokens[i][2:]
    else:
        answer += " " + tokens[i]

In [26]:
def question_answer(question, text):
    
    #tokenize question and text in ids as a pair
    input_ids = tokenizer(question, text, return_tensors="tf")
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids.input_ids[0])
    
    #model output using input_ids and segment_ids
    output = model(input_ids)
    
    #reconstructing the answer
    answer_start = tf.argmax(tf.cast(output.start_logits, tf.int32), axis=1)
    answer_end = tf.where(tf.equal(output.end_logits, float(tf.reduce_max(output.end_logits[0]))))[:,-1]

    if answer_end >= answer_start:
        answer = tokens[int(answer_start)]
        for i in range(int(answer_start)+1, int(answer_end)+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nAnswer:\n{}".format(answer.capitalize()))

In [27]:
text = """
New York (CNN) -- More than 80 Michael Jackson collectibles -- including the late pop star's famous rhinestone-studded glove from a 1983 performance -- were 
auctioned off Saturday, reaping a total $2 million. Profits from the auction at the Hard Rock Cafe in New York's Times Square crushed pre-sale expectations of 
only $120,000 in sales. The highly prized memorabilia, which included items spanning the many stages of Jackson's career, came from more than 30 fans, 
associates and family members, who contacted Julien's Auctions to sell their gifts and mementos of the singer. Jackson's flashy glove was the big-ticket item 
of the night, fetching $420,000 from a buyer in Hong Kong, China. Jackson wore the glove at a 1983 performance during \"Motown 25,\" an NBC special where he 
debuted his revolutionary moonwalk. Fellow Motown star Walter \"Clyde\" Orange of the Commodores, who also performed in the special 26 years ago, said he 
asked for Jackson's autograph at the time, but Jackson gave him the glove instead. "The legacy that [Jackson] left behind is bigger than life for 
me,\" Orange said. \"I hope that through that glove people can see what he was trying to say in his music and what he said in his music.\" Orange said 
he plans to give a portion of the proceeds to charity. Hoffman Ma, who bought the glove on behalf of Ponte 16 Resort in Macau, paid a 25 percent buyer's 
premium, which was tacked onto all final sales over $50,000. Winners of items less than $50,000 paid a 20 percent premium.
"""

question = "Where was the Auction held?"


In [28]:
type(text), type(question)


(str, str)

In [29]:
question_answer(question, text)


Answer:
The hard rock cafe in new york ' s times square


In [30]:
print("Original answer:\n", data.loc[data["question"] == question]["answer"].values[0])

Original answer:
 Hard Rock Cafe


In [31]:
question="How is called the place where was done the Auction?"
question_answer(question, text)


Answer:
Hard rock cafe


In [32]:
text2 = """
Seth Rollins won the WWE World Heavyweight Championship in the main event at Wrestlemania 31 in the Levi's stadium that attended 
over 100000 people, in a fantastic cashing of The Money In The Bank contract in a heartstopping moment where both fighter were
absolutely hurted. The Undertaker made his return and continued succesfully his streak by defeating Bray Wyatt in a tough match.
John Cena defeated Rusev for the US Championship in a match that became into a battle of nations and 
at moments polemic. One of the dream match all fans loved to see is the battle of two of the greatest teams in the industry: NWO vs DX, 
being represented by Sting and Triple H respectively.
Vince McMahon (WWE's chairman) said: The event was a success and left satisfied the fans because of the surprise factor and the perfect performance
of all superstars. Nevertheless there are plans for tomorrow at monday night RAW where the former champion Brock Lesnar
will have a rematch for the title. """

In [33]:
questions = ["Who won WWE championship?",
             "How many people attented Wrestlemania 31?",
             "Who made his return at the event?",
             "Who represented NWO?",
             "Who represented DX?",
             "Who is WWE's chairman?",
             "What title won John Cena?",
             "When is will be monday night RAW?",
             "Why the event was a success?",
             "What will happen at RAW live event?"]

In [34]:
for quest in questions:
    question_answer(quest, text2)


Answer:
Seth rollins

Answer:
Over 100000

Answer:
The undertaker

Answer:
Sting

Answer:
Triple h

Answer:
Vince mcmahon

Answer:
Us championship

Answer:
Tomorrow

Answer:
The surprise factor and the perfect performance of all superstars

Answer:
Brock lesnar will have a rematch for the title


In [36]:
text = input("Please enter your text: \n")
question = input("\nPlease enter your question: \n")

while True:
    question_answer(question, text)
    
    flag = True
    flag_N = False
    
    while flag:
        response = input("\nDo you want to ask another question based on this text (Y/N)? ")
        if response[0] == "Y":
            question = input("\nPlease enter your question: \n")
            flag = False
        elif response[0] == "N":
            print("\nBye!")
            flag = False
            flag_N = True
            
    if flag_N == True:
        break

Please enter your text: 
MAINAK

Please enter your question: 
Who is WWE's chairman?

Answer:
Mainak

Do you want to ask another question based on this text (Y/N)? Who was John Cena?

Do you want to ask another question based on this text (Y/N)? When did Brock Lesnar rematch

Do you want to ask another question based on this text (Y/N)? N

Bye!
